# Oasis Reinsurance Test Tool

This notebook allows example reinsurance structures to be input in OED format and ran against the development version of the Oasis finanical engine. 

## Oasis Reinsurance FM v0.1

* Basic direct insurance modelling
    * Buildings, Other buildings contents and BI
    * Blanket limit
    * Blanket deductible
    * Site limit
    * Site deductible
* Location FAC treaties
    * No occurrence limit
* Cat XL treaties

In [1]:
# Step 1 - general Python code set up
%config IPCompleter.greedy=True
import reinsurance_tester
import jupyter_helper
import pandas as pd

In [2]:
# Step 2 - load the OED for a worked example, in this case FM Test Case 23.
# Note that only the currently used fields are shown unless show_all is set to True. 
(account_df, location_df, ri_info_df, ri_scope_df, do_reinsurance) = reinsurance_tester.load_oed_dfs('simple_QS', show_all=False)

In [3]:
# Step 3 - view/edit the account data. 
account_grid = jupyter_helper.show_df(account_df)
account_grid

UWdyaWRXaWRnZXQoZ3JpZF9vcHRpb25zPXsnaGlnaGxpZ2h0U2VsZWN0ZWRSb3cnOiBUcnVlLCAnZnVsbFdpZHRoUm93cyc6IFRydWUsICdyb3dIZWlnaHQnOiAyOCwgJ2VuYWJsZUNvbHVtblLigKY=


In [11]:
# Step 4 - view/edit the location data.
location_grid = jupyter_helper.show_df(location_df)
location_grid

UWdyaWRXaWRnZXQoZ3JpZF9vcHRpb25zPXsnaGlnaGxpZ2h0U2VsZWN0ZWRSb3cnOiBUcnVlLCAnZnVsbFdpZHRoUm93cyc6IFRydWUsICdyb3dIZWlnaHQnOiAyOCwgJ2VuYWJsZUNvbHVtblLigKY=


In [12]:
# Step 5 - view/edit the reinsurance info data.
ri_info_grid = jupyter_helper.show_df(ri_info_df)
ri_info_grid

UWdyaWRXaWRnZXQoZ3JpZF9vcHRpb25zPXsnaGlnaGxpZ2h0U2VsZWN0ZWRSb3cnOiBUcnVlLCAnZnVsbFdpZHRoUm93cyc6IFRydWUsICdyb3dIZWlnaHQnOiAyOCwgJ2VuYWJsZUNvbHVtblLigKY=


In [13]:
# Step 6 - view/edit the reinsurance scope data.
ri_scope_grid = jupyter_helper.show_df(ri_scope_df)
ri_scope_grid

UWdyaWRXaWRnZXQoZ3JpZF9vcHRpb25zPXsnaGlnaGxpZ2h0U2VsZWN0ZWRSb3cnOiBUcnVlLCAnZnVsbFdpZHRoUm93cyc6IFRydWUsICdyb3dIZWlnaHQnOiAyOCwgJ2VuYWJsZUNvbHVtblLigKY=


In [14]:
# Step 7 - pick up any edits in the grid before running the analysis
account_df = account_grid.get_changed_df()
location_df = location_grid.get_changed_df()
ri_info_df = ri_info_grid.get_changed_df()
ri_scope_df = ri_scope_grid.get_changed_df()

In [15]:
# Step 8 - run the OED data though the Oasis Financial Module and output the losses by item at each inuring level.
net_losses = reinsurance_tester.run_test('test', account_df, location_df, ri_info_df, ri_scope_df, loss_factor=1.0, do_reinsurance=do_reinsurance)
print("Ran {} inuring layers".format(len(net_losses) - 1))
print("Losses for:")
for key in net_losses.keys():
    print("\t{}".format(key))

Ran 1 inuring layers
Losses for:
	Direct
	Inuring priority:1 - Risk level:SEL


In [16]:
# Step 9 - view the direct losses.
net_losses['Direct']

,policy_number,account_number,location_number,coverage_type_id,peril_id,tiv,loss_gul,loss_il
0,1,1,1,1,1,1000,1000.0,50.0
1,1,1,1,3,1,500,500.0,25.0
2,1,1,1,4,1,500,500.0,25.0
3,1,1,2,1,1,1000,1000.0,50.0
4,1,1,2,3,1,500,500.0,25.0
5,1,1,2,4,1,500,500.0,25.0
6,2,2,3,1,1,1000,1000.0,50.0
7,2,2,3,3,1,500,500.0,25.0
8,2,2,3,4,1,500,500.0,25.0
9,2,2,4,1,1,1000,1000.0,50.0


In [17]:
# Step 10 - view the losses for the first inuring layer.
net_losses['Inuring priority:1 - Risk level:SEL']

,policy_number,account_number,location_number,coverage_type_id,peril_id,tiv,loss_pre,loss_net
0,1,1,1,1,1,1000,50.0,25.0
1,1,1,1,3,1,500,25.0,12.5
2,1,1,1,4,1,500,25.0,12.5
3,1,1,2,1,1,1000,50.0,25.0
4,1,1,2,3,1,500,25.0,12.5
5,1,1,2,4,1,500,25.0,12.5
6,2,2,3,1,1,1000,50.0,25.0
7,2,2,3,3,1,500,25.0,12.5
8,2,2,3,4,1,500,25.0,12.5
9,2,2,4,1,1,1000,50.0,25.0
